## Author Shmakov Maxim

In [1]:
from sys import executable
!{executable} -m pip install -U numpy pandas

Requirement already up-to-date: numpy in /usr/local/lib/python3.7/site-packages (1.18.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.7/site-packages (1.0.3)


In [2]:
from os import listdir

import numpy as np
import pandas as pd

/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
csv_file = [file for file in listdir('.') if file.endswith('csv')][0]

df = pd.read_csv(csv_file)
df.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Life Boat,Survived
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,2,Yes
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,11,Yes
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No


## I. Start with basic EDA (Exploratory data analysis): 
1) Compute average `Age` of passengers and number of passengers who survived and not survived grouped by `Sex` and `Passenger Class` (24 numbers);

2) What can you say about survivors based on the resulting table (open question), e.g. what is the surviving ratio for females in First class compared to the Second and Third? 
This answer is limited to 150 words.

3) What is the average number of males and females on all boats (rounded to the closest integer)? 
Do not forget to filter out all `?` in `Life Boat` attribute. 


In [4]:
df.groupby(['Sex', 'Passenger Class', 'Survived'])['Age'].agg(['mean', 'size'])

mean  size
Sex    Passenger Class Survived                 
Female First           No        35.200000     5
                       Yes       37.109375   139
       Second          No        34.090909    12
                       Yes       26.711051    94
       Third           No        23.418750   110
                       Yes       20.814815   106
Male   First           No        43.658163   118
                       Yes       36.168240    61
       Second          No        33.092593   146
                       Yes       17.449274    25
       Third           No        26.679598   418
                       Yes       22.436441    75

In [5]:
df_by_size = df[~df['Life Boat'].isna()].groupby(['Life Boat', 'Sex']).agg('size')
df_by_size.groupby('Sex').agg('mean')

Sex
Female    13.869565
Male       6.958333
dtype: float64

The amount of survived women is higher than survived men in each passenger class. First class passengers are older than passengers from all of other classes. Higher class means higher probability of surviving probably due to higher level of service (96% of women of first class survived). 

## II. Proceed with feature generation.

1) Drop the column `Life Boat`.

2) Generate new attribute `Family size`: sum up `No of Parents or Children on Board` and `No of Siblings or Spouses on Board` and add 1 (for passenger himself, thanks to @pianovanastya). What is the average family size? In which class did the biggest family travel?
In this case, isn’t it better to group people not by ticket number, but by the family size? Then we can divide the number of people with the same family size by the family size value and receive the number of families for each family size.
Do not drop original attributes.

3) It seems that `Passenger Fare` is total among all passengers with the same `Ticket Number`: create new attribute `Single passenger fare`. For every passenger you need to compute the number of passengers with the same `Ticket Number` and then use this number as a divisor for `Passenger Fare`. 
Do not drop the original attribute.

4) Impute missing values: for numerical attributes use averaging over three groups: `Passenger Class`, `Sex`, `Embarkation Port`; for every numerical attribute create separate column that contain 1 for imputed value and 0 for originally presented. 
This step is mainly for practicing your groupby/join skills. In real tasks this kind of imputation is relatively rare.

5) Pre-process categorical attributes: For every categorical attribute create a separate column that contains 1 for a missing value and 0 for originally presented. One-hot encode categorical attributes with less than 20 unique values, drop other categorical attributes; drop original (that you pre-processed during this step) attributes. 

6) Set the role of the `Survived` attribute to `label`.


In [6]:
df_LB = df.drop(columns=['Life Boat'])
df_LB.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Survived
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,Yes
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,Yes
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,No
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,No
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,No


In [7]:
df_LB['Family size'] = df_LB['No of Siblings or Spouses on Board'] + df_LB['No of Parents or Children on Board'] + 1
df_LB.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Survived,Family size
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,Yes,1
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,Yes,4
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4


In [8]:
f"{df_LB['Family size'].mean():.2f}"

'1.88'

In [9]:
list(df_LB[df_LB['Family size'] == max(df_LB['Family size'])]['Passenger Class'].unique())

['Third']

In [10]:
tickets = df_LB.groupby('Ticket Number')['Ticket Number'].transform('size')
df_LB['Single passenger fare'] = df_LB['Passenger Fare'] / tickets
df_LB.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Survived,Family size,Single passenger fare
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,Yes,1,52.834375
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,Yes,4,25.258333
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4,25.258333
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4,25.258333
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4,25.258333


In [11]:
df_LB.groupby(['Passenger Class'])['Single passenger fare'].mean().reset_index()

,Passenger Class,Single passenger fare
0,First,33.910500
1,Second,11.411010
2,Third,7.329146


In [12]:
# Select numeric columns to iterate through them
for column in df_LB.select_dtypes(include=[np.number]).columns:
    if df_LB[column].isna().any():
        df_LB[column + ' imputed'] = df_LB[column].isna().astype(int)

        df_LB[column] = df_LB[column].fillna(
            df_LB.groupby(['Sex', 'Passenger Class', 'Port of Embarkation'])
            [column].transform('mean'))

df_LB.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Survived,Family size,Single passenger fare,Age imputed,Passenger Fare imputed,Single passenger fare imputed
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,Yes,1,52.834375,0,0,0
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,Yes,4,25.258333,0,0,0
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4,25.258333,0,0,0
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4,25.258333,0,0,0
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,No,4,25.258333,0,0,0


In [13]:
print(df_LB["Age imputed"].any(),
      df_LB["Passenger Fare imputed"].any(),
      df_LB["Single passenger fare imputed"].any())

True True True


In [14]:
# Same for categorical variables
categ_vars = df_LB.select_dtypes(include=['object']).columns

for column in categ_vars:
    if df_LB[column].isna().any():
        df_LB[column + ' missing'] = df_LB[column].isna().astype(int)

df_LB = df_LB.drop(columns=categ_vars[df_LB[categ_vars].nunique() >= 20])
df_LB.head()

,Passenger Class,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Passenger Fare,Port of Embarkation,Survived,Family size,Single passenger fare,Age imputed,Passenger Fare imputed,Single passenger fare imputed,Cabin missing,Port of Embarkation missing
0,First,Female,29.0000,0,0,211.3375,Southampton,Yes,1,52.834375,0,0,0,0,0
1,First,Male,0.9167,1,2,151.5500,Southampton,Yes,4,25.258333,0,0,0,0,0
2,First,Female,2.0000,1,2,151.5500,Southampton,No,4,25.258333,0,0,0,0,0
3,First,Male,30.0000,1,2,151.5500,Southampton,No,4,25.258333,0,0,0,0,0
4,First,Female,25.0000,1,2,151.5500,Southampton,No,4,25.258333,0,0,0,0,0


In [18]:
pd.get_dummies(df_LB, drop_first=True).head()

,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Passenger Fare,Family size,Single passenger fare,Age imputed,Passenger Fare imputed,Single passenger fare imputed,Cabin missing,Port of Embarkation missing,Passenger Class_Second,Passenger Class_Third,Sex_Male,Port of Embarkation_Queenstown,Port of Embarkation_Southampton,Survived_Yes
0,29.0000,0,0,211.3375,1,52.834375,0,0,0,0,0,0,0,0,0,1,1
1,0.9167,1,2,151.5500,4,25.258333,0,0,0,0,0,0,0,1,0,1,1
2,2.0000,1,2,151.5500,4,25.258333,0,0,0,0,0,0,0,0,0,1,0
3,30.0000,1,2,151.5500,4,25.258333,0,0,0,0,0,0,0,1,0,1,0
4,25.0000,1,2,151.5500,4,25.258333,0,0,0,0,0,0,0,0,0,1,0
